In [1]:
import pandas as pd 
import tensorflow as tf 
import numpy as np 
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.preprocessing import MinMaxScaler
from sklearn.utils import shuffle
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
import pandas as pd
import matplotlib.pyplot as plt
tf.random.set_seed(42) 
from alive_progress import alive_bar
import time
from keras import backend as K
import pickle as pkl
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier

categorical_cross_entropy = tf.keras.losses.CategoricalCrossentropy()

# Models of defender 

In [2]:
defender_dataset = pd.read_csv('./dataset/Bot_IoT/defender_dataset.csv')

In [3]:
data_temp_defender_dataset = defender_dataset
data_temp_defender_dataset = data_temp_defender_dataset.drop(['attack'],axis=1)
defender_dataset=data_temp_defender_dataset

In [4]:
X_train_defender, X_test_defender, y_train_defender, y_test_defender = train_test_split(defender_dataset.drop(columns = ["category"]), defender_dataset.category, stratify=defender_dataset.category, shuffle=True, test_size=0.25,random_state=42)

In [ ]:
scaler = MinMaxScaler()
scaler.fit(X_train_defender.to_numpy()) # to numpy to avoid the warning later when we predict with a numpy instead of dataframe
X_train_defender_scaled = scaler.transform(X_train_defender)
X_test_defender_scaled = scaler.transform(X_test_defender)  # normalize test set on training set

### DNN (Defender)

In [ ]:
# Model 1 initialization and training ---  DNN (Defender)
# One-hot encoding and convertion into tensors because TensorFlow DNN
y_train_defender_ohe = pd.get_dummies(y_train_defender)
y_train_defender_ohe_tf = tf.convert_to_tensor(y_train_defender_ohe, np.float32)

y_test_defender_ohe = pd.get_dummies(y_test_defender)
y_test_defender_ohe_tf = tf.convert_to_tensor(y_test_defender_ohe, np.float32)

output_number = 5
eval_metric = 'categorical_accuracy'
activ_out = 'softmax'
neurons_number = 256
lr = 0.001
features_number = X_train_defender_scaled.shape[1]

model = tf.keras.Sequential([
    tf.keras.layers.Dense(neurons_number, input_shape=(features_number,), activation="relu"),
    tf.keras.layers.Dense(neurons_number, activation="relu"),
    tf.keras.layers.Dense(neurons_number, activation="relu"),
    tf.keras.layers.Dense(output_number, activation=activ_out)
])
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=lr),
    loss = categorical_cross_entropy,
    metrics=[eval_metric]
)

model.fit(x=X_train_defender_scaled, y=y_train_defender_ohe_tf, epochs=10, batch_size=100, verbose=1)
model.evaluate(x=X_test_defender_scaled, y=y_test_defender_ohe_tf, verbose=1)

# Model saving 
tf.keras.models.save_model(model, './models_defender/dnndefender')
# Uncomment all before if you already trained the model
# Model Loading
# model = tf.keras.models.load_model('./models_defender/dnnattacker, custom_objects={'categorical_cross_entropy': categorical_cross_entropy})

# Evaluation
y_pred = model.predict(X_test_defender_scaled)
y_pred_vect = np.argmax(y_pred,1)
print(classification_report(y_true=y_test_defender, y_pred=y_pred_vect))

### Random Forest (Defender)

In [ ]:
# Model2 Initialization and training - Random Forest (Defender)
model2 = RandomForestClassifier(n_estimators = 200, n_jobs = -1, random_state=0)
# No need to scale data as in Neural Network
model2.fit(X_train_defender, y_train_defender)

# Model saving 
pkl.dump(model2, open('./models_defender/rfdefender.pkl', 'wb'))

# Uncomment all before if you already trained the model
# Model Loading
# model2 = pkl.load(open('./models_defender/rfattacker.pkl', 'rb')

# Evaluation
pred = model2.predict(X_test_defender)
print( classification_report(y_true=y_test_defender, y_pred=pred))

### ADABoost (Defender)

In [ ]:
# Model 3  ADABoost (Defender)
model3 = AdaBoostClassifier(n_estimators=400, learning_rate=0.5, random_state=0)
model3.fit(X_train_defender, y_train_defender)
# Model saving 
pkl.dump(model3, open('./models_defender/adaboostdefender.pkl', 'wb'))

# Uncomment all before if you already trained the model
# Model Loading
# model3 = pkl.load(open('./models_defender/adaboostattacker.pkl', 'rb'))

# Evaluation
pred = model3.predict(X_test_defender)
print(classification_report(y_true=y_test_defender, y_pred=pred))

### KNN (Defender)

In [ ]:
# Model 4 - KNN (Defender)
model4 = KNeighborsClassifier(n_neighbors=3)
model4.fit(X_train_defender, y_train_defender)
# Model saving 
pkl.dump(model4, open('./models_defender/knndefender.pkl', 'wb'))
# Uncomment all before if you already trained the model
# Model Loading
# model4 = pkl.load(open('./models_defender/knnattacker.pkl', 'rb'))

# Evaluation
pred = model4.predict(X_test_defender)
print(classification_report(y_true=y_test_defender, y_pred=pred))

### Decision Tree (Defender)

In [ ]:
# Model 5 -  Decision Tree (Defender)
model5 = DecisionTreeClassifier(criterion="entropy", random_state=0)
model5.fit(X_train_defender, y_train_defender)
# Model saving 
pkl.dump(model5, open('./models_defender/dtdefender.pkl', 'wb'))
# Uncomment all before if you already trained the model
# Model Loading
# model5 = pkl.load(open('./models_defender/dtattacker.pkl', 'rb'))

# Evaluation
pred = model5.predict(X_test_defender)
print(classification_report(y_true=y_test_defender, y_pred=pred))

### xgboost  (Defender)

In [ ]:
# Model 6  -  xgboost  (Defender)
model6 = XGBClassifier(n_estimators=200, max_depth=5, learning_rate=0.5, objective='multi:softprob')
model6.fit(X_train_defender, y_train_defender)
# Model saving 
pkl.dump(model6, open('./models_defender/xgboostdefender.pkl', 'wb'))
# Uncomment all before if you already trained the model
# Model Loading
# model6 = pkl.load(open('./models_defender/xgboostattacker.pkl', 'rb'))
preds = model6.predict(X_test_defender)
print( classification_report(y_true=y_test_defender, y_pred=preds))

# Models of Attacker

In [13]:
attacker_dataset = pd.read_csv('./dataset/Bot_IoT/attacker_dataset.csv')
attacker_dataset=attacker_dataset.drop(columns = ['attack'])

In [14]:
X_train_attacker, X_test_attacker, y_train_attacker, y_test_attacker = train_test_split(attacker_dataset.drop(columns = ["category"]), attacker_dataset.category, stratify=attacker_dataset.category, shuffle=True, test_size=0.25,random_state=42)

In [ ]:
scaler = MinMaxScaler()
scaler.fit(X_train_attacker.to_numpy()) # to numpy to avoid the warning later when we predict with a numpy instead of dataframe
X_train_attacker_scaled = scaler.transform(X_train_attacker)
X_test_attacker_scaled = scaler.transform(X_test_attacker)  # normalize test set on training set

### DNN (Attacker)

In [ ]:
# Model 1 initialization and training ---  DNN (Attacker)
# One-hot encoding and convertion into tensors because TensorFlow DNN
y_train_attacker_ohe = pd.get_dummies(y_train_attacker)
y_train_attacker_ohe_tf = tf.convert_to_tensor(y_train_attacker_ohe, np.float32)

y_test_attacker_ohe = pd.get_dummies(y_test_attacker)
y_test_attacker_ohe_tf = tf.convert_to_tensor(y_test_attacker_ohe, np.float32)

output_number = 5
eval_metric = 'categorical_accuracy'
activ_out = 'softmax'
neurons_number = 128
lr = 0.001
features_number = X_train_attacker_scaled.shape[1]

model = tf.keras.Sequential([
    tf.keras.layers.Dense(neurons_number, input_shape=(features_number,), activation="relu"),
    tf.keras.layers.Dense(neurons_number, activation="relu"),
    tf.keras.layers.Dense(neurons_number, activation="relu"),
    tf.keras.layers.Dense(neurons_number, activation="relu"),
    tf.keras.layers.Dense(output_number, activation=activ_out)
])
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=lr),
    loss = categorical_cross_entropy,
    metrics=[eval_metric]
)

model.fit(x=X_train_attacker_scaled, y=y_train_attacker_ohe_tf, epochs=10, batch_size=100, verbose=1)
model.evaluate(x=X_test_attacker_scaled, y=y_test_attacker_ohe_tf, verbose=1)

# Model saving 
tf.keras.models.save_model(model, './models_attacker/dnnattacker')
# Uncomment all before if you already trained the model
# Model Loading
# model = tf.keras.models.load_model('./models_attacker/dnnattacker, custom_objects={'categorical_cross_entropy': categorical_cross_entropy})

# Evaluation
y_pred = model.predict(X_test_attacker_scaled)
y_pred_vect = np.argmax(y_pred,1)
print(classification_report(y_true=y_test_attacker, y_pred=y_pred_vect))

### Random Forest (Attacker)

In [ ]:
# Model2 Initialization and training - Random Forest (Attacker)
model2 = RandomForestClassifier(n_estimators = 300, n_jobs = -1, random_state=0)
# No need to scale data as in Neural Network
model2.fit(X_train_attacker, y_train_attacker)

# Model saving 
pkl.dump(model2, open('./models_attacker/rfattacker.pkl', 'wb'))
# Uncomment all before if you already trained the model
# Model Loading
# model2 = pkl.load(open('./models_attacker/rfattacker.pkl', 'rb'))

# Evaluation
pred = model2.predict(X_test_attacker)
print(classification_report(y_true=y_test_attacker, y_pred=pred))

### ADABoost (Attacker)

In [ ]:
# Model 3 - ADABoost (Attacker)
model3 = AdaBoostClassifier(n_estimators=300, learning_rate=0.5, random_state=0)
model3.fit(X_train_attacker, y_train_attacker)
# Model saving 
pkl.dump(model3, open('./models_attacker/adaboostattacker.pkl', 'wb'))
# Uncomment all before if you already trained the model
# Model Loading
# model3 = pkl.load(open('./models_attacker/adaboostattacker.pkl', 'rb'))

# Evaluation
pred = model3.predict(X_test_attacker)
print(classification_report(y_true=y_test_attacker, y_pred=pred))

### KNN (Attacker)

In [ ]:
# Model 4 - KNN (Attacker)
model4 = KNeighborsClassifier(n_neighbors=5)
model4.fit(X_train_attacker, y_train_attacker)
# Model saving 
pkl.dump(model4, open('./models_attacker/knnattacker.pkl', 'wb'))
# Uncomment all before if you already trained the model
# Model Loading
# model4 = pkl.load(open('./models_attacker/knnattacker.pkl', 'rb'))

# Evaluation
pred = model4.predict(X_test_attacker)
print(classification_report(y_true=y_test_attacker, y_pred=pred))

### Decision Tree (Attacker)

In [ ]:
# Model 5 -  Decision Tree (Attacker)
model5 = DecisionTreeClassifier(criterion="gini", random_state=0)
model5.fit(X_train_attacker, y_train_attacker)
# Model saving 
pkl.dump(model5, open('./models_attacker/dtattacker.pkl', 'wb'))
# Uncomment all before if you already trained the model
# Model Loading
# model5 = pkl.load(open('./models_attacker/dtattacker.pkl', 'rb'))

# Evaluation
pred = model5.predict(X_test_attacker)
print(classification_report(y_true=y_test_attacker, y_pred=pred))

### xgboost  (Attacker)

In [ ]:
# Model 6  -  xgboost  (Attacker)
model6 = XGBClassifier(n_estimators=300, max_depth=5, learning_rate=0.5, objective='multi:softprob')
model6.fit(X_train_attacker, y_train_attacker)
# Model saving 
pkl.dump(model6, open('./models_attacker/xgboostattacker.pkl', 'wb'))
# Uncomment all before if you already trained the model
# Model Loading
# model6 = pkl.load(open('./models_attacker/xgboostattacker.pkl', 'rb'))

# Evaluation
preds = model6.predict(X_test_attacker)
print(classification_report(y_true=y_test_attacker, y_pred=preds))